In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_tfidf, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test_tfidf)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    # Detect the error type
    predicted_label = rf_model.predict(text_tfidf)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Correct the sentence using the T5 model
    corrected_sentence = correct_grammar(text, t5_model, t5_tokenizer)
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')
Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       0.67      0.22      0.33         9
           1       0.67      0.36      0.47        11
           2       0.00      0.00      0.00        11
           3       0.08      0.09      0.08        22
           4       0.00      0.00      0.00         6
           5       0.19      0.33      0.24         9
           6       0.43      0.30      0.35        10
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00        12
           9       0.83      0.42      0.56        12
          10       1.00      0.08      0.15        12
          11       0.00      0.00      0.00        15
          12       0.00      0.00      0.00         9
          13       0.78      0.78      0.78         9
          14       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Special tokens have been added in the vocabulary, make sure the 

Enter a sentence for error detection and correction (or type 'exit' to quit): i go to the store everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I go to the store everyday.
Enter a sentence for error detection and correction (or type 'exit' to quit): i goes to the store everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I go to the store every day.
Enter a sentence for error detection and correction (or type 'exit' to quit): i wanting a cofee.
Detected Error Type: Spelling Mistakes
Corrected Sentence: I want a room.
Enter a sentence for error detection and correction (or type 'exit' to quit): i wanting a coffee.
Detected Error Type: Spelling Mistakes
Corrected Sentence: I want a coffee.
Enter a sentence for error detection and correction (or type 'exit' to quit): i think the code is done.
Detected Error Type: Spelling Mistakes
Corrected Sentence: I think the code is done.
Enter a sentence for error detection and correction (or type 'exit' to quit):

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Correct the sentence using the T5 model
    corrected_sentence = correct_grammar(text, t5_model, t5_tokenizer)
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       1.00      0.11      0.20         9
           1       0.80      0.36      0.50        11
           2       0.00      0.00      0.00        11
           3       0.08      0.09      0.09        22
           4       0.14      0.67      0.24         6
           5       0.25      0.44      0.32         9
           6       0.50      0.30      0.37        10
           7       0.50      0.14      0.22         7
           8       0.20      0.08      0.12        12
           9       0.75      0.50      0.60        12
          10       1.00      0.17      0.29        12
          11       0.50      0.07      0.12        15
          12       0.00      0.00      0.00         9
          13       0.86      0.67      0.75         9
          14       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Special tokens have been added in the vocabulary, make sure the 

Enter a sentence for error detection and correction (or type 'exit' to quit): exit


In [ ]:
import re
import numpy as np
import pandas as pd
import heapq
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Dijkstra's algorithm for finding the shortest path in the graph of sentence transformations
def dijkstra(graph, start, end):
    queue, visited = [(0, start, [])], set()
    heapq.heapify(queue)
    while queue:
        (cost, node, path) = heapq.heappop(queue)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return cost, path
        visited.add(node)
        for (next_node, c) in graph[node]:
            if next_node not in visited:
                heapq.heappush(queue, (cost + c, next_node, path))
    return float("inf"), []

# Create a graph of possible sentence transformations
def create_transformation_graph(sentence, model, tokenizer):
    graph = defaultdict(list)
    corrected_sentence = correct_grammar(sentence, model, tokenizer)
    # For simplicity, assume each transformation has a cost of 1
    graph[sentence].append((corrected_sentence, 1))
    return graph

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Create a graph of possible sentence transformations
    graph = create_transformation_graph(text, t5_model, t5_tokenizer)
    # Find the shortest path using Dijkstra's algorithm
    cost, path = dijkstra(graph, text, graph[text][0][0])
    corrected_sentence = path[-1] if path else text
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       0.33      0.11      0.17         9
           1       0.67      0.36      0.47        11
           2       0.00      0.00      0.00        11
           3       0.18      0.18      0.18        22
           4       0.16      0.50      0.24         6
           5       0.31      0.44      0.36         9
           6       0.44      0.40      0.42        10
           7       0.25      0.14      0.18         7
           8       0.00      0.00      0.00        12
           9       0.71      0.42      0.53        12
          10       0.20      0.08      0.12        12
          11       0.50      0.07      0.12        15
          12       0.00      0.00      0.00         9
          13       0.71      0.56      0.63         9
          14       0.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Enter a sentence for error detection and correction (or type 'exit' to quit): I is goes to the stores everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I goes to the stores everyday.
Enter a sentence for error detection and correction (or type 'exit' to quit): I is goes to the store everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I goes to the store everyday.
Enter a sentence for error detection and correction (or type 'exit' to quit): exit


In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    return error_type, text

# Collect all detected errors
errors = []
# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, original_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    errors.append((detected_error_type, original_sentence))

# Display all detected errors
print("\nDetected Errors:")
for i, (error_type, original_sentence) in enumerate(errors, start=1):
    print(f"{i}. Error Type: {error_type}, Sentence: {original_sentence}")


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       1.00      0.11      0.20         9
           1       0.80      0.36      0.50        11
           2       0.00      0.00      0.00        11
           3       0.14      0.14      0.14        22
           4       0.14      0.67      0.24         6
           5       0.23      0.33      0.27         9
           6       0.50      0.30      0.37        10
           7       0.50      0.14      0.22         7
           8       0.33      0.08      0.13        12
           9       0.75      0.50      0.60        12
          10       1.00      0.17      0.29        12
          11       0.50      0.07      0.12        15
          12       0.00      0.00      0.00         9
          13       0.75      0.67      0.71         9
          14       0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Special tokens have been added in the vocabulary, make sure the 

Enter a sentence for error detection and correction (or type 'exit' to quit): exit

Detected Errors:


In [ ]:
import re
import numpy as np
import pandas as pd
import heapq
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Dijkstra's algorithm for finding the shortest path in the graph of sentence transformations
def dijkstra(graph, start, end):
    queue, visited = [(0, start, [])], set()
    heapq.heapify(queue)
    while queue:
        (cost, node, path) = heapq.heappop(queue)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return cost, path
        visited.add(node)
        for (next_node, c) in graph[node]:
            if next_node not in visited:
                heapq.heappush(queue, (cost + c, next_node, path))
    return float("inf"), []

# Create a graph of possible sentence transformations
def create_transformation_graph(sentence, model, tokenizer):
    graph = defaultdict(list)
    corrected_sentence = correct_grammar(sentence, model, tokenizer)
    # For simplicity, assume each transformation has a cost of 1
    graph[sentence].append((corrected_sentence, 1))
    return graph

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans,
                               t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error types
    predicted_labels = rf_model.predict(text_features)
    error_types = label_encoder.inverse_transform(predicted_labels)
    # Initialize lists to store detected errors and corrected sentences
    detected_errors = []
    corrected_sentences = []
    for error_type in error_types:
        # Check if the sentence has no errors based on some threshold
        if error_type == "No_Error":  # Assuming "No_Error" is the label for "no error"
            detected_errors.append("No error in the sentence")
            corrected_sentences.append(text)
            continue
        # Create a graph of possible sentence transformations
        graph = create_transformation_graph(text, t5_model, t5_tokenizer)
        # Find the shortest path using Dijkstra's algorithm
        cost, path = dijkstra(graph, text, graph[text][0][0])
        corrected_sentence = path[-1] if path else text
        detected_errors.append(error_type)
        corrected_sentences.append(corrected_sentence)
    return detected_errors, corrected_sentences

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_types, corrected_sentences = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    for error_type, corrected_sentence in zip(detected_error_types, corrected_sentences):
        print(f"Detected Error Type: {error_type}")
        print(f"Corrected Sentence: {corrected_sentence}")


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters found:  {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       0.67      0.22      0.33         9
           1       0.57      0.36      0.44        11
           2       0.10      0.09      0.10        11
           3       0.17      0.23      0.19        22
           4       0.17      0.17      0.17         6
           5       0.33      0.44      0.38         9
           6       0.44      0.40      0.42        10
           7       0.10      0.14      0.12         7
           8       0.00      0.00      0.00        12
           9       0.62      0.67      0.64        12
          10       0.17      0.08      0.11        12
          11       0.25      0.07      0.11        15
          12       0.00      0.00      0.00         9
          13  

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Enter a sentence for error detection and correction (or type 'exit' to quit): exit


In [ ]:
import re
import numpy as np
import pandas as pd
import heapq
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Dijkstra's algorithm for finding the shortest path in the graph of sentence transformations
def dijkstra(graph, start, end):
    queue, visited = [(0, start, [])], set()
    heapq.heapify(queue)
    while queue:
        (cost, node, path) = heapq.heappop(queue)
        if node in visited:
            continue
        path = path + [node]
        if node == end:
            return cost, path
        visited.add(node)
        for (next_node, c) in graph[node]:
            if next_node not in visited:
                heapq.heappush(queue, (cost + c, next_node, path))
    return float("inf"), []

# Create a graph of possible sentence transformations
def create_transformation_graph(sentence, model, tokenizer):
    graph = defaultdict(list)
    corrected_sentence = correct_grammar(sentence, model, tokenizer)
    # For simplicity, assume each transformation has a cost of 1
    graph[sentence].append((corrected_sentence, 1))
    return graph

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Create a graph of possible sentence transformations
    graph = create_transformation_graph(text, t5_model, t5_tokenizer)
    # Find the shortest path using Dijkstra's algorithm
    cost, path = dijkstra(graph, text, graph[text][0][0])
    corrected_sentence = path[-1] if path else text
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 30}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       0.50      0.11      0.18         9
           1       0.80      0.36      0.50        11
           2       0.00      0.00      0.00        11
           3       0.14      0.09      0.11        22
           4       0.27      0.67      0.38         6
           5       0.00      0.00      0.00         9
           6       0.50      0.30      0.37        10
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00        12
           9       0.83      0.42      0.56        12
          10       1.00      0.08      0.15        12
          11       0.00      0.00      0.00        15
          12       0.00      0.00      0.00         9
          13       0.86      0.67      0.75         9
          14       0.25

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Special tokens have been added in the vocabulary, make sure the 

Enter a sentence for error detection and correction (or type 'exit' to quit): exit


In [4]:
import re
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Uncomment these lines if you haven't downloaded the nltk data before
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load sample data
data = pd.read_csv("/content/Grammar Correction.csv")  # Replace with actual dataset path

# Print column names to verify
print("Column names in the dataset:", data.columns)

# Check if the necessary columns are present
if 'Ungrammatical_Statement' not in data.columns or 'Error_Type' not in data.columns:
    raise KeyError("The dataset must contain 'Ungrammatical_Statement' and 'Error_Type' columns")

# Preprocessing function
def preprocess_text(Ungrammatical_Statement):
    # Lowercase conversion
    # Ungrammatical_Statement = Ungrammatical_Statement.lower()
    # Remove special characters
    Ungrammatical_Statement = re.sub(r'\W', ' ', Ungrammatical_Statement)
    # Tokenization
    words = word_tokenize(Ungrammatical_Statement)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing
data['cleaned_text'] = data['Ungrammatical_Statement'].apply(preprocess_text)

# Label encoding (assuming binary classification for errors)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Error_Type'])

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Apply K-means clustering
num_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
data['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Add cluster labels as features
X = np.hstack((tfidf_matrix.toarray(), data['cluster'].values.reshape(-1, 1)))
y = data['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate the best model on the test data
y_pred_rf = best_rf_model.predict(X_test)
print("Random Forest Model evaluation:")
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")

# Load the pre-trained T5 model and tokenizer fine-tuned for grammar correction
model_name = 'vennify/t5-base-grammar-correction'  # Known model for grammar correction
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to correct grammatical errors using the T5 model
def correct_grammar(text, model, tokenizer):
    input_text = "gec: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Function to detect errors and correct the sentence
def detect_and_correct_errors(text, rf_model, vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder):
    # Preprocess the text for error detection
    cleaned_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([cleaned_text])
    text_cluster = kmeans.predict(text_tfidf)[0]
    text_features = np.hstack((text_tfidf.toarray(), np.array([[text_cluster]])))
    # Detect the error type
    predicted_label = rf_model.predict(text_features)
    # Check if the sentence has no errors based on some threshold
    if predicted_label == 0:  # Assuming 0 is the label for "no error"
        return "No error in the sentence", text
    error_type = label_encoder.inverse_transform(predicted_label)[0]
    # Correct the sentence using the T5 model
    corrected_sentence = correct_grammar(text, t5_model, t5_tokenizer)
    return error_type, corrected_sentence

# Take user input for testing the model
while True:
    user_input = input("Enter a sentence for error detection and correction (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    detected_error_type, corrected_sentence = detect_and_correct_errors(user_input, best_rf_model, tfidf_vectorizer, kmeans, t5_model, t5_tokenizer, label_encoder)
    print(f"Detected Error Type: {detected_error_type}")
    print(f"Corrected Sentence: {corrected_sentence}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Column names in the dataset: Index(['Serial_Number', 'Error_Type', 'Ungrammatical_Statement',
       'Standard_English'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
Random Forest Model evaluation:
              precision    recall  f1-score   support

           0       0.25      0.11      0.15         9
           1       0.67      0.36      0.47        11
           2       0.00      0.00      0.00        11
           3       0.14      0.14      0.14        22
           4       0.18      0.50      0.26         6
           5       0.36      0.44      0.40         9
           6       0.60      0.30      0.40        10
           7       0.33      0.14      0.20         7
           8       0.00      0.00      0.00        12
           9       0.67      0.50      0.57        12
          10       0.14      0.08      0.11        12
          11       0.00      0.00      0.00        15
          12       0.00      0.00      0.00         9
          13       0.75      0.67      0.71         9
          14       0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Special tokens have been added in the vocabulary, make sure the 

Enter a sentence for error detection and correction (or type 'exit' to quit): i goes to the store everyday.
Detected Error Type: Verb Tense Errors
Corrected Sentence: I go to the store every day.
Enter a sentence for error detection and correction (or type 'exit' to quit): I wanting a book.
Detected Error Type: Spelling Mistakes
Corrected Sentence: I want a book.
Enter a sentence for error detection and correction (or type 'exit' to quit): they eating a pizza for lunch.
Detected Error Type: Verb Tense Errors
Corrected Sentence: They are eating a pizza for lunch.
Enter a sentence for error detection and correction (or type 'exit' to quit): He is an inteeligent boy.
Detected Error Type: Negation Errors
Corrected Sentence: He is an intelligent boy.
Enter a sentence for error detection and correction (or type 'exit' to quit): He is an intelligent boy.
Detected Error Type: Negation Errors
Corrected Sentence: He is an intelligent boy.
Enter a sentence for error detection and correction (or t